In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Configuration
NUM_USERS = 500
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Plan configurations
PLANS = {
    'Free': {'price': 0, 'features': 'basic'},
    'Starter': {'price': 29, 'features': 'standard'},
    'Pro': {'price': 99, 'features': 'advanced'},
    'Enterprise': {'price': 299, 'features': 'full'}
}

# User statuses
USER_STATUSES = ['active', 'inactive', 'suspended', 'deleted']
SUBSCRIPTION_STATUSES = ['active', 'canceled', 'past_due', 'trialing']

print("Generating User Accounts vs Billing Records dataset...")

Generating User Accounts vs Billing Records dataset...


In [2]:
# ==============================================================================
# GENERATE USER ACCOUNTS (Product Database)
# ==============================================================================
print("\n1. Generating user accounts...")

users = []
for i in range(1, NUM_USERS + 1):
    signup_date = START_DATE + timedelta(days=random.randint(0, 330))
    plan = random.choice(list(PLANS.keys()))
    status = random.choice(['active'] * 80 + ['inactive'] * 15 + ['suspended'] * 4 + ['deleted'] * 1)
    
    # Last login logic
    if status == 'active':
        days_since_login = random.randint(0, 7)
    elif status == 'inactive':
        days_since_login = random.randint(30, 180)
    else:
        days_since_login = random.randint(90, 365)
    
    last_login = END_DATE - timedelta(days=days_since_login)
    
    users.append({
        'user_id': f'usr_{i:05d}',
        'email': f'user{i}@example.com',
        'account_status': status,
        'plan_type': plan,
        'features_enabled': PLANS[plan]['features'],
        'signup_date': signup_date,
        'last_login': last_login if last_login >= signup_date else signup_date,
        'account_created_date': signup_date
    })

users_df = pd.DataFrame(users)


1. Generating user accounts...


In [3]:
# ==============================================================================
# GENERATE BILLING SUBSCRIPTIONS (Payment Processor)
# ==============================================================================
print("2. Generating billing subscriptions...")

subscriptions = []
subscription_id = 1

for user in users:
    user_id = user['user_id']
    email = user['email']
    plan = user['plan_type']
    signup_date = user['signup_date']
    
    # Free users shouldn't have subscriptions (mostly)
    if plan == 'Free':
        # But 5% do (billing error - should be canceled)
        if random.random() < 0.05:
            subscriptions.append({
                'subscription_id': f'sub_{subscription_id:05d}',
                'customer_email': email,
                'user_id': user_id,
                'plan': plan,
                'status': random.choice(['active', 'canceled']),
                'billing_amount': 0,
                'start_date': signup_date,
                'current_period_end': END_DATE + timedelta(days=30)
            })
            subscription_id += 1
        continue
    
    # Paid users should have subscriptions
    # But 10% don't (free riders - discrepancy!)
    if random.random() < 0.10:
        continue
    
    # 15% have wrong plan in billing (discrepancy!)
    if random.random() < 0.15:
        wrong_plans = [p for p in PLANS.keys() if p != plan and p != 'Free']
        billing_plan = random.choice(wrong_plans) if wrong_plans else plan
    else:
        billing_plan = plan
    
    # Determine subscription status
    if user['account_status'] == 'active':
        sub_status = random.choice(['active'] * 85 + ['past_due'] * 10 + ['trialing'] * 5)
    elif user['account_status'] == 'inactive':
        sub_status = random.choice(['canceled'] * 60 + ['active'] * 30 + ['past_due'] * 10)
    else:
        sub_status = random.choice(['canceled'] * 80 + ['active'] * 20)
    
    subscriptions.append({
        'subscription_id': f'sub_{subscription_id:05d}',
        'customer_email': email,
        'user_id': user_id,
        'plan': billing_plan,
        'status': sub_status,
        'billing_amount': PLANS[billing_plan]['price'],
        'start_date': signup_date,
        'current_period_end': END_DATE + timedelta(days=30) if sub_status in ['active', 'trialing', 'past_due'] else signup_date + timedelta(days=random.randint(30, 300))
    })
    subscription_id += 1

subscriptions_df = pd.DataFrame(subscriptions)

2. Generating billing subscriptions...


In [5]:
# ==============================================================================
# CREATE INTENTIONAL DISCREPANCIES
# ==============================================================================
print("3. Creating intentional discrepancies...")

# DISCREPANCY 1: Ghost subscriptions (billing exists, no user account)
# 3% of subscriptions
num_ghosts = int(len(subscriptions_df) * 0.03)
print(f"   - Adding {num_ghosts} ghost subscriptions (billing without user)")

for i in range(num_ghosts):
    ghost_plan = random.choice(['Starter', 'Pro', 'Enterprise'])
    subscriptions.append({
        'subscription_id': f'sub_{subscription_id:05d}',
        'customer_email': f'ghost{i}@example.com',
        'user_id': f'usr_99{i:03d}',  # Non-existent user ID
        'plan': ghost_plan,
        'status': 'active',
        'billing_amount': PLANS[ghost_plan]['price'],
        'start_date': START_DATE + timedelta(days=random.randint(0, 300)),
        'current_period_end': END_DATE + timedelta(days=30)
    })
    subscription_id += 1

# DISCREPANCY 2: Duplicate subscriptions (same user, multiple active subs)
# 2% of users
num_duplicates = int(len(users_df) * 0.02)
print(f"   - Adding {num_duplicates} duplicate subscriptions (same user, multiple billing)")

duplicate_users = random.sample(list(users_df[users_df['plan_type'] != 'Free']['user_id']), num_duplicates)
for user_id in duplicate_users:
    user_data = users_df[users_df['user_id'] == user_id].iloc[0]
    dup_plan = random.choice(['Starter', 'Pro'])
    subscriptions.append({
        'subscription_id': f'sub_{subscription_id:05d}',
        'customer_email': user_data['email'],
        'user_id': user_id,
        'plan': dup_plan,
        'status': 'active',
        'billing_amount': PLANS[dup_plan]['price'],
        'start_date': user_data['signup_date'] + timedelta(days=random.randint(30, 200)),
        'current_period_end': END_DATE + timedelta(days=30)
    })
    subscription_id += 1

# DISCREPANCY 3: Payment failed but user still has access
# Already handled in user status vs subscription status mismatch

# DISCREPANCY 4: Trial expired but still active
# 5% of trials
trial_subs = subscriptions_df[subscriptions_df['status'] == 'trialing'].copy()
num_expired_trials = int(len(trial_subs) * 0.15)
print(f"   - Creating {num_expired_trials} expired trials with active access")

subscriptions_df = pd.DataFrame(subscriptions)


3. Creating intentional discrepancies...
   - Adding 10 ghost subscriptions (billing without user)
   - Adding 10 duplicate subscriptions (same user, multiple billing)
   - Creating 2 expired trials with active access


In [ ]:
# ==============================================================================
# CELL 3: Explore the Data
# ==============================================================================
print("\n" + "=" * 80)
print("STEP 2: DATA EXPLORATION")
print("=" * 80)

print("\n📊 User Accounts Summary:")
print(users_df.groupby(['account_status', 'plan_type']).size().unstack(fill_value=0))

print("\n💳 Billing Subscriptions Summary:")
print(subscriptions_df.groupby(['status', 'plan']).size().unstack(fill_value=0))

# Visualize user distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

users_df['account_status'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Users by Account Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Status')
axes[0].set_ylabel('Count')

users_df['plan_type'].value_counts().plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Users by Plan Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Plan')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()


In [6]:
# ==============================================================================
# SAVE DATA
# ==============================================================================
print("\n4. Saving data files...")

#users_df.to_csv('user_accounts.csv', index=False)
users_df.to_csv( r"C:\Users\hp\Desktop\folder for data output\user_accounts.csv", index=False)
#subscriptions_df.to_csv('billing_subscriptions.csv', index=False)
subscriptions_df.to_csv( r"C:\Users\hp\Desktop\folder for data output\billing_subscriptions.csv", index=False)


print(f"\n✓ Generated {len(users_df)} user accounts")
print(f"✓ Generated {len(subscriptions_df)} billing subscriptions")


4. Saving data files...

✓ Generated 500 user accounts
✓ Generated 385 billing subscriptions


In [7]:
# ==============================================================================
# SUMMARY STATISTICS
# ==============================================================================
print("\n" + "=" * 40)
print("DATASET SUMMARY")
print("=" * 40)

print("\nUser Accounts Breakdown:")
print(users_df.groupby(['account_status', 'plan_type']).size().unstack(fill_value=0))

print("\nBilling Subscriptions Breakdown:")
print(subscriptions_df.groupby(['status', 'plan']).size().unstack(fill_value=0))

# Calculate expected discrepancies
active_paid_users = len(users_df[(users_df['account_status'] == 'active') & (users_df['plan_type'] != 'Free')])
active_subs = len(subscriptions_df[subscriptions_df['status'].isin(['active', 'trialing'])])

print("\n" + "=" * 40)
print("INTENTIONAL DISCREPANCIES FOR RECONCILIATION")
print("=" * 40)

print(f"""
1. FREE RIDERS (Users with access but no billing):
   - ~10% of paid users have no subscription
   - These users are using paid features without paying
   
2. GHOST SUBSCRIPTIONS (Billing without user account):
   - {num_ghosts} subscriptions with no corresponding user
   - Money being collected but no service delivered
   
3. PLAN MISMATCHES (User has Pro access but billed for Starter):
   - ~15% of paid users have different plans in app vs billing
   - Either over-delivering or under-charging
   
4. STATUS MISMATCHES (Active user but canceled subscription):
   - Users with canceled/past_due billing but still have access
   - Payment failures not enforced
   
5. DUPLICATE SUBSCRIPTIONS (One user, multiple active subs):
   - {num_duplicates} users being billed multiple times
   - Double charging issue
   
6. FREE PLAN WITH BILLING (Free users with active subscriptions):
   - ~5% of Free users have billing records
   - Should be canceled
   
7. DELETED USERS WITH ACTIVE BILLING:
   - Users deleted their accounts but still being charged
   - Refund liability
""")

print("\nFiles created:")
print("- user_accounts.csv (product database)")
print("- billing_subscriptions.csv (payment processor)")
print("\nYour reconciliation should catch all of these issues!")


DATASET SUMMARY

User Accounts Breakdown:
plan_type       Enterprise  Free  Pro  Starter
account_status                                
active                  94    91  101      101
deleted                  1     0    2        3
inactive                19    17   28       17
suspended                5    10    6        5

Billing Subscriptions Breakdown:
plan      Enterprise  Free  Pro  Starter
status                                  
active            82     1   99       99
canceled          16     1   21       12
past_due          10     0   17       12
trialing           1     0    8        6

INTENTIONAL DISCREPANCIES FOR RECONCILIATION

1. FREE RIDERS (Users with access but no billing):
   - ~10% of paid users have no subscription
   - These users are using paid features without paying
   
2. GHOST SUBSCRIPTIONS (Billing without user account):
   - 10 subscriptions with no corresponding user
   - Money being collected but no service delivered
   
3. PLAN MISMATCHES (User has Pro